In [6]:
from procDataSet import TrainingQuery
from bert_serving.client import BertClient
from multiprocessing import Pool
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim  as optim
import argparse
from keras.utils import to_categorical
import pandas as pd
import numpy as np
from torch.functional import F
import os
import jieba
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import clear_output

jieba.load_userdict(os.path.join('data', 'dict.txt.big'))
[jieba.add_word(i, freq=None, tag=None) for i in ['不支持','文林苑', '都更案','十八趴','證所稅','前瞻建設']]

[None, None, None, None, None, None]

In [ ]:
top_num = 300

In [12]:
with open('data/stop_word_zhtw.txt') as file:
    data = file.read()
    
stop_words = data.split('\n')
stop_words += ['「', '」', '，', '\n', '）', '（', ')', '(']

In [13]:
from sklearn.model_selection import train_test_split
folder = 'data/'
raw_training_data = pd.read_csv(os.path.join(folder,'TD.csv'))
news_urls = pd.read_csv(os.path.join(folder,'NC_1.csv'))
contents = pd.read_json(os.path.join(folder,'url2content.json'), typ=pd.Series)

## sort the contents by index
keys, content_list = contents.keys(), contents.values

In [14]:
test_query = np.array(pd.read_csv('./data/QS_1.csv').Query)

In [ ]:
for test_id , text_q in enumerate(test_query):
    print(jieba.lcut_for_search(text_q))

In [15]:
import multiprocessing

NumberCPU = multiprocessing.cpu_count()
jieba.initialize()

def jbcut(x):
    if x is not None:
        sen = jieba.lcut(x, cut_all=False)
        sen = [i for i in sen if i not in stop_words]
        return sen
    else:
        return None

import jieba.posseg as pseg
def psegcut(x):
    if x is not None:
        sen = pseg.lcut(x)
        sen = [i for i in sen if i not in stop_words]
        return sen
    else:
        return None
    
pool = multiprocessing.Pool(processes=NumberCPU)
sentenece_arr = pool.map(psegcut,content_list)
pool.close()
pool.join()

## okapi model

In [22]:
from gensim.summarization import bm25
bm25Model = bm25.BM25(sentenece_arr)

In [ ]:
stands = ['支持', '應該', '反對', '贊成', '不'] 

In [16]:
train_query = list(set(raw_training_data.Query))

y_train = []
y_index = {}
for i  in train_query:
    index = np.where(raw_training_data.Query==i)
    data = raw_training_data.iloc[index]
    y = dict(zip(data.News_Index,data.Relevance))
    y_idx = [ (int(idx.split('_')[1])-1, rel )  for idx, rel in zip(data.News_Index, data.Relevance)]
    y_train.append(y)
    y_index[i] = [y_idx]

In [17]:
common_query = set(train_query) & set(test_query)
common_query = list(common_query)
common = [ train_query.index(i) for i in common_query]

In [18]:
[ len(list(y_train[i].values())) for i in common]

[230, 95, 139, 40, 230]

In [19]:
[ np.sum(list(y_train[i].values())) / 300 for i in range(20)]

[2.1933333333333334,
 1.0533333333333332,
 0.5766666666666667,
 1.3233333333333333,
 1.4466666666666668,
 0.9433333333333334,
 0.2833333333333333,
 0.23,
 0.21333333333333335,
 2.296666666666667,
 0.6933333333333334,
 0.77,
 0.7066666666666667,
 1.3,
 1.3733333333333333,
 0.24333333333333335,
 0.14,
 1.35,
 2.6466666666666665,
 2.7866666666666666]

In [45]:
np.mean(total_scores)

0.4789999999999999

In [20]:
[ np.sum(list(y_train[i].values())) / 300 for i in common]

[0.9433333333333334, 0.23, 0.6933333333333334, 0.14, 1.35]

In [ ]:
total_scores = list()
for test_id , text_q in enumerate(train_query):
    text = jieba.lcut(text_q)
    text = [ t for t in text if t not in stop_words]
    scores = np.array(bm25Model.get_scores(text))
    top_query = np.argsort(scores)[::-1][:100]
    irrelevant_query = np.argsort(scores)[:100]
    
    print(test_id ,text_q, text)
    print("top query num {}".format(len(top_query)))
    
    
    for query in top_query:
        all_words = [ (sentenece_arr[query][cnt], bm25Model.get_score(sentenece_arr[query], cnt))
                     for cnt, i in enumerate(sentenece_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        text += top_words
    scores += 0.3 * np.array(bm25Model.get_scores(text))
    
    irrelevant = []
    for query in irrelevant_query:
        all_words = [ (sentenece_arr[query][cnt], bm25Model.get_score(sentenece_arr[query], cnt))
                     for cnt, i in enumerate(sentenece_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        irrelevant += top_words
    
    scores -= 0.3 * np.array(bm25Model.get_scores(irrelevant))
#     for idx, rel in (y_index[text_q][0]):
#         scores[idx] += np.mean(scores) * idx
    
    top_num = 300
    keys = pd.DataFrame(np.argsort(scores)[::-1][:top_num])
    ans = keys[0].apply(lambda x: 'news_{:06d}'.format(x+1))
    
    validation_score = 0
    for a in ans:
        if a in y_train[test_id].keys():
            validation_score += y_train[test_id][a]
            
    total_scores.append(validation_score/top_num)
    print("{} / {} ".format(validation_score,(np.sum(list(y_train[test_id].values())))))

0 堅決反對政府舉債發展前瞻建設計畫 ['堅決', '反對', '政府', '舉債', '發展', '前瞻建設', '計畫']
top query num 100
304 / 658 
1 核四應該啟用 ['核四', '應該', '啟用']
top query num 100
45 / 316 
2 十二年國教高中職「免學費補助」適用對象增加是不對的 ['十二年', '國教', '高中職', '免', '學費', '補助', '適用', '對象', '增加', '不對']
top query num 100
114 / 173 
3 遠雄大巨蛋工程應停工或拆除 ['遠雄', '大巨蛋', '工程', '應', '停工', '拆除']
top query num 100
229 / 397 
4 贊成流浪動物零撲殺 ['贊成', '流浪', '動物', '零', '撲殺']
top query num 100
299 / 434 
5 年金改革應取消或應調降軍公教月退之優存利率十八趴 ['年金', '改革', '應', '取消', '應', '調降', '軍公教', '月', '退之', '優存', '利率', '十八趴']
top query num 100
101 / 283 
6 贊成文林苑都更案可依法拆除王家 ['贊成', '文林苑', '都更案', '依法', '拆除', '王家']
top query num 100
63 / 85 
7 油價應該凍漲或緩漲 ['油價', '應該', '凍漲', '緩漲']
top query num 100


$${\displaystyle k_{1}\in [1.2,2.0]}, {\displaystyle b=0.75}$$

In [ ]:
total_scores = list()
for test_id , text_q in enumerate(test_query):
    text = jieba.lcut(text_q)
    text = [ t for t in text if t not in stop_words]
    scores = bm25Model.get_scores(text)
    top_query = np.argsort(scores)[::-1][:100]
    irrelevant_query = np.argsort(scores)[:100]
    
    print(test_id ,text_q, text)
    print("top query num {}".format(len(top_query)))
    
    for query in top_query:
        all_words = [ (sentenece_arr[query][cnt], bm25Model.get_score(sentenece_arr[query], cnt))
                     for cnt, i in enumerate(sentenece_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        text += top_words
    scores += 0.3 * np.array(bm25Model.get_scores(text))
    
    irrelevant = []
    for query in irrelevant_query:
        all_words = [ (sentenece_arr[query][cnt], bm25Model.get_score(sentenece_arr[query], cnt))
                     for cnt, i in enumerate(sentenece_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        irrelevant += top_words
    
    scores -= 0.3 * np.array(bm25Model.get_scores(irrelevant))
    
#     scores = bm25Model.get_scores(text)
    delta = np.max(scores) 
    if text_q in y_index.keys():
        for idx, rel in (y_index[text_q][0]):
            scores[idx] += delta * idx
            
    total_scores.append(scores)

0 通姦在刑法上應該除罪化 ['通姦', '刑法', '應該', '除罪化']
top query num 100
1 應該取消機車強制二段式左轉(待轉) ['應該', '取消', '機車', '強制', '二段式', '左轉', '待轉']
top query num 100
2 支持博弈特區在台灣合法化 ['支持', '博弈', '特區', '台灣', '合法化']
top query num 100
3 中華航空空服員罷工是合理的 ['中華', '航空', '空服員', '罷工', '合理']
top query num 100
4 性交易應該合法化 ['性交易', '應該', '合法化']
top query num 100
5 ECFA早收清單可（有）達到其預期成效 ['ECFA', '早', '收清單', '達到', '預期', '成效']
top query num 100
6 應該減免證所稅 ['應該', '減免', '證所稅']
top query num 100
7 贊成中油在觀塘興建第三天然氣接收站 ['贊成', '中油', '觀塘', '興建', '第三', '天然氣', '接收站']
top query num 100
8 支持中國學生納入健保 ['支持', '中國', '學生', '納入', '健保']
top query num 100
9 支持臺灣中小學（含高職、專科）服儀規定（含髮、襪、鞋）給予學生自主 ['支持', '臺灣', '中小學', '含', '高職', '專科', '服儀', '規定', '含髮', '襪', '鞋', '給予', '學生', '自主']
top query num 100
10 不支持使用加密貨幣 ['不支持', '使用', '加密', '貨幣']
top query num 100
11 不支持學雜費調漲 ['不支持', '學雜費', '調漲']
top query num 100
12 同意政府舉債發展前瞻建設計畫 ['同意', '政府', '舉債', '發展', '前瞻建設', '計畫']
top query num 100
13 支持電競列入體育競技 ['支持', '電競', '列入', '體育競技']
top query num 100
14 反對台鐵東移徵收案 ['反對', '台鐵', '東

## Test the model

In [34]:
top_num = 300
search_result = np.zeros((20,top_num))
for cnt,i in enumerate(total_scores):
    keys = np.argsort(i)[::-1][:top_num]
    search_result[cnt] += keys
    
search_result = search_result.astype(np.int)

df = pd.DataFrame()
df['Query_Index'] = ['q_{:02d}'.format(i+1) for i in range(20)]

for i in range(top_num):
    df['Rank_{:03d}'.format(i+1)] = search_result[:, i] + 1

for i, row in df.iterrows():
    df.iloc[i, 1:] = df.iloc[i, 1:].apply(lambda x: 'news_{:06d}'.format(x))

In [35]:
import os
sorted(os.listdir('output/'))

['simple.csv',
 'simple10.csv',
 'simple11.csv',
 'simple12.csv',
 'simple13.csv',
 'simple14.csv',
 'simple15.csv',
 'simple16.csv',
 'simple17.csv',
 'simple18.csv',
 'simple19.csv',
 'simple2.csv',
 'simple20.csv',
 'simple21.csv',
 'simple3.csv',
 'simple4.csv',
 'simple5.csv',
 'simple8.csv',
 'simple9.csv',
 'top2000.csv']

In [36]:
fname = 'simple22.csv'
df.to_csv('output/' + fname, index=False)

# search_result[0]